In [24]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *


product = 'com.dbs.in.digitalbank'
driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')
timeOut = 30 # seconds

# driver.set_page_load_timeout(timeOut)
# wait = WebDriverWait( driver, timeOut)
driver.implicitly_wait(timeOut) 
url = 'https://play.google.com/store/apps/details?id=' + product + '&hl=en_IN&showAllReviews=true'
driver.get(url)
page = driver.page_source
soup_expatistan = BeautifulSoup(page, "html.parser")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
downloads = soup_expatistan.find('span', {'class': 'AYi5wd'}) 
print('Total comments are: ', downloads.text)
 
ratingDiv = soup_expatistan.find('div', {'class': 'pf5lIe'}) 

import re
text = str(ratingDiv.get_text)
try:
    found = re.search('Rated(.+?)stars out of five stars', text).group(1).strip()
except AttributeError:
    found = 'None' 
print('Raitings: ', found)

Total comments are:  79,921
Raitings:  4.1


In [26]:
# expatistan_table = soup_expatistan.find("h1", class_="AHFaub")
# print(expatistan_table.string)

In [186]:
# no_of_reviews = 1000
# ratings = soup_expatistan.find("span", class_="EymY4b")
# print(ratings.text)

In [27]:
# SCROLL_PAUSE_TIME = 2.0

# # Get scroll height
# last_height = driver.execute_script("return document.body.scrollHeight")
# count=0

# while True:
#     # Scroll down to bottom
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # Wait to load page
#     time.sleep(SCROLL_PAUSE_TIME)

#     # Calculate new scroll height and compare with last scroll height
# #     new_height = driver.execute_script("return document.body.scrollHeight")
#     if count == 4:
#         try:
#             show = driver.find_element_by_class_name('PFAhAf')
#             show.click()
#             count = 0
#         except NoSuchElementException:
#             print('got exception ', NoSuchElementException)
#     else:
#         count = count + 1
    

In [1]:
from selenium.common.exceptions import NoSuchElementException, WebDriverException, TimeoutException
from IPython.display import display, clear_output



requiredComments = 15000

count = 0
totalComments = 200
while totalComments < requiredComments:
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     driver.execute_script("window.scrollTo(0, -10000);")
    
    if count >= 4:
        try:
            show = driver.find_element_by_class_name('PFAhAf')
            show.click()
            totalComments = totalComments + 160
            count = 0
        except NoSuchElementException:
            print('got exception ', NoSuchElementException)
    else:
        count = count + 1

    time.sleep(1)
    
#     clear_output(wait=True)
    print('Fetched comments: %d/%d' %(totalComments, requiredComments))
    


In [29]:
print('Trying to extract comments from driver, please wait for sometime...')

xpath = "/html/body/div[1]/div[5]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]"
div = driver.find_element_by_xpath(xpath).get_attribute("innerHTML")
soup = BeautifulSoup(div, 'html.parser')
allCommentDiv = soup.find("div", {"jsname": "fk8dgd"}).find_all("div", {"jscontroller": "H6eOGe"})

print('total comments are: ', len(allCommentDiv))

info = []
count = 0
for each in allCommentDiv:
    name = each.find('span', {'class': 'X43Kjb'})
    ratings = len(each.find_all('div', {'class': 'vQHuPe'}))
    date = each.find('span', {'class': 'p2TkOb'})
    comment = each.find('span', {'jsname': 'bN97Pc'})
    fillComment =  each.find('span', {'jsname': 'fbQN7e'})
    if len(fillComment.text) > 0 :
        comment = fillComment
        
    clear_output(wait=True)
    display('Total extrated comments are: ' + str(count))
    count = count + 1
#     print('name: ', name.text)
#     print('rating: ', ratings)
#     print('date: ', date.text)
#     print('comment: ', comment.text)
    result = [name.text, date.text, comment.text, ratings]
    info.append(result)
    
reverseList = reversed(info)

'Total extrated comments are: 15079'

In [30]:
print(len(info))
print(info[720])

15080
['AKHILESH SHARMA', '18 April 2019', 'ok', 5]


In [31]:
backup = info

In [32]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data=np.array(info), columns=['name', 'date', 'comment', 'rating'])
fileNameToStore = 'android_comments_jan_24April2019.pkl'
df.to_pickle(fileNameToStore)
b = pd.read_pickle(fileNameToStore)

In [42]:
# merged_df = pd.concat([old_df, new_df], sort=True)
# merged_df = pd.merge(old_df, new_df, on='datetime')